### 사용자 정의 모델 클래스
- 부모 클래스 : nn.Module
- 필수 오버라이딩 : 
    * \__init__() : 모델 층 구성 즉, 설계
    * forward() : 순방향 학습 진행 코드 구현

In [27]:
# 모듈 로딩
import torch                                    # 텐서 관련 모듈
import torch.nn as nn                           # 인공신경망 관련 모듈
import torch.nn.functional as F                 # 인공신경망 관련 함수들 모듈 (손실함수, 활성화함수 등)
import torch.optim as optim                     # 최적화 관련 모듈 (가중치, 절편 빠르게 찾아주는 알고리즘)
from torchinfo import summary                   # 모델 구조 및 정보 관련 모듈
from torchmetrics.regression import *           # 회귀 성능 지표 관련 모듈
from torchmetrics.classification import *       # 분류 성능 지표 관련 모듈

In [28]:
# 랜덤 고정
torch.manual_seed(1)

# 텐서 저장 및 실행 위치 설정
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

- [기본] 신경망 클래스 <hr>
    * 입력층 - 입력 : 피쳐수 고정
    * 출력층 - 출력 : 타겟수 고정
    * 은닉층 - 고정

In [29]:
# 모델 설계
# 데이터셋 : 피쳐 4개, 타겟 1개, 회귀
# 입 력 층 : 입력   4개    출력  20개   AF ReLU
# 은 닉 층 : 입력  20개    출력 100개   AF ReLU
# 출 력 층 : 입력 100개    출력   1개   AF X, (분류일때는 Sigmoid & Softmax)

# 회귀일때는 출력층의 활성화함수가 필요 없이 그대로 값을 출력하면 됨

In [44]:
# 입력 피쳐 수가 4개로 고정인 모델
class MyModel(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(4, 20) # W 4 + b 1 => 1P, 5*20 = 100개 변수
        self.hidden_layer = nn.Linear(20, 100) # W 20 + b 1 => 1P, 21*100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1) # W 100 + b 1 => 1P, 101*1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4+b (총 20개 만듦)
        y = F.relu(y)               # 0 <= y ----> 죽은 relu ==> leakyReLU

        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1+x2W2+...+x20W20+b (총 100개 만듦)
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1W1+x2W2+...+x100W100+b (총 1개 만듦)

In [40]:
# 입력 피쳐 수가 동적인 모델
class MyModel2(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, 20) # W 4 + b 1 => 1P, 5*20 = 100개 변수
        self.hidden_layer = nn.Linear(20, 100) # W 20 + b 1 => 1P, 21*100 = 2100개 변수
        self.output_layer = nn.Linear(100, 1) # W 100 + b 1 => 1P, 101*1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4+b (총 20개 만듦)
        y = F.relu(y)               # 0 <= y ----> 죽은 relu ==> leakyReLU

        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1+x2W2+...+x20W20+b (총 100개 만듦)
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1W1+x2W2+...+x100W100+b (총 1개 만듦)

In [45]:
# 입력 피쳐 수, 은닉층 퍼셉트론 수 동적인 모델
class MyModel3(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, in_out, h_out):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.input_layer = nn.Linear(in_features, in_out)   # W 4 + b 1 => 1P, 5*20 = 100개 변수
        self.hidden_layer = nn.Linear(in_out, h_out)        # W 20 + b 1 => 1P, 21*100 = 2100개 변수
        self.output_layer = nn.Linear(h_out, 1)             # W 100 + b 1 => 1P, 101*1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4+b (총 20개 만듦)
        y = F.relu(y)               # 0 <= y ----> 죽은 relu ==> leakyReLU

        y = self.hidden_layer(y)    # 1개 퍼셉트론 : y = x1W1+x2W2+...+x20W20+b (총 100개 만듦)
        y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1W1+x2W2+...+x100W100+b (총 1개 만듦)

In [57]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, hidden_list):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.hidden_list = hidden_list
        self.hidden_layer_cnt = len(hidden_list)

        self.input_layer = nn.Linear(in_features, hidden_list[0])   # W 4 + b 1 => 1P, 5*20 = 100개 변수

        # hidden_layer 자리는 forward 쪽으로 뺌

        self.output_layer = nn.Linear(hidden_list[self.hidden_layer_cnt-1], 1)    # W 100 + b 1 => 1P, 101*1 = 101개 변수

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4+b (총 20개 만듦)
        y = F.relu(y)               # 0 <= y ----> 죽은 relu ==> leakyReLU

        for i in range(self.hidden_layer_cnt):
            self.hidden_layer = nn.Linear(self.hidden_list[i], self.hidden_list[i+1])
            y = self.hidden_layer(y)
            y = F.relu

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1W1+x2W2+...+x100W100+b (총 1개 만듦)

In [114]:
# 은닉층의 개수가 동적인 모델
class MyModel4(nn.Module):
    # 인스턴스/객체 생성 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    def __init__(self, in_features, hidden_list):
        # 부모 클래스 생성
        super().__init__()
        # 자식 클래스의 인스턴스 속성 설정
        self.hidden_list = hidden_list
        self.hidden_layer_cnt = len(hidden_list)-1

        self.input_layer = nn.Linear(in_features, hidden_list[0])   # W 4 + b 1 => 1P, 5*20 = 100개 변수
        # print(f'input layer : {self.input_layer}')

        # 히든 레이어
        self.hidden_layer_list = nn.ModuleList()
        for i in range(self.hidden_layer_cnt):
            self.hidden_layer_list.append(nn.Linear(self.hidden_list[i], self.hidden_list[i+1]))
            # print(f'hidden layer{i+1} : {self.hidden_layer_list[i]}')

        self.output_layer = nn.Linear(hidden_list[self.hidden_layer_cnt], 1)    # W 100 + b 1 => 1P, 101*1 = 101개 변수
        # print(f'output layer : {self.output_layer}')

    # 순방향/전방향 학습 진행 시 자동 호출되는 메서드 (콜백함수 Callback func : 시스템에서 호출되는 함수)
    # 전달 인자 : 학습용 데이터셋
    def forward(self, x):
        print('calling forward()')
        y = self.input_layer(x)     # 1개 퍼셉트론 : y = x1W1 + x2W2 + x3W3 + x4W4+b (총 20개 만듦)
        y = F.relu(y)               # 0 <= y ----> 죽은 relu ==> leakyReLU

        for i in self.hidden_layer_list:
            y = i(y)
            y = F.relu(y)

        return self.output_layer(y) # 1개 퍼셉트론 : y = x1W1+x2W2+...+x100W100+b (총 1개 만듦)

In [115]:
# 모델 인스턴스 생성
m1 = MyModel2(in_features=3)

In [118]:
m4 = MyModel4(in_features=3, hidden_list=[100, 200, 300, 400])
m4

MyModel4(
  (input_layer): Linear(in_features=3, out_features=100, bias=True)
  (hidden_layer_list): ModuleList(
    (0): Linear(in_features=100, out_features=200, bias=True)
    (1): Linear(in_features=200, out_features=300, bias=True)
    (2): Linear(in_features=300, out_features=400, bias=True)
  )
  (output_layer): Linear(in_features=400, out_features=1, bias=True)
)

In [113]:
# 모델 파라미터 즉, W와 b 확인
for m in m1.named_parameters(): print(m)

('input_layer.weight', Parameter containing:
tensor([[-0.5341,  0.4648, -0.5534],
        [ 0.1510, -0.0716, -0.4107],
        [-0.2082, -0.2223, -0.5268],
        [-0.4919,  0.3113, -0.3583],
        [-0.1807, -0.1970, -0.2683],
        [-0.3693, -0.5101, -0.4672],
        [-0.0986,  0.0320,  0.2292],
        [-0.0751, -0.4333,  0.4379],
        [ 0.0347,  0.1997, -0.4950],
        [-0.5715, -0.0840,  0.0107],
        [-0.0485,  0.1111, -0.5635],
        [ 0.5011,  0.2536, -0.3822],
        [ 0.2695, -0.3871, -0.5035],
        [-0.3660, -0.0831, -0.4610],
        [ 0.0271,  0.1010, -0.4564],
        [-0.4992,  0.5024,  0.4691],
        [ 0.4357,  0.4206,  0.3831],
        [-0.2931, -0.4852, -0.1086],
        [-0.3810,  0.4570,  0.3342],
        [ 0.2351,  0.2713,  0.3153]], requires_grad=True))
('input_layer.bias', Parameter containing:
tensor([ 0.2114, -0.0610,  0.4379,  0.0813,  0.0749, -0.0525, -0.1201,  0.2234,
        -0.1872,  0.4069,  0.1871,  0.0035,  0.5260, -0.0927,  0.5683,

In [91]:
# 학습 진행 ==> 모델인스턴스명(데이터)
# 임의의 데이터
dataTS = torch.FloatTensor([[1, 3, 5], [2, 4, 6]])
targetTS = torch.FloatTensor([[4],[5]])

# 학습
pred_y = m1(dataTS) # m1.forward(dataTS)로 해도 됨

print(pred_y)

calling forward()
tensor([[0.3860],
        [0.5733]], grad_fn=<AddmmBackward0>)
